In [ ]:
import pandas as pd
from river_framework import (
    RiverFlowDataProcessor,
    RiverFlowModel,
    PATH_DF,
)

In [ ]:
set_list = ["Train", "Validation", "Combined", "Test"]
metric_list = [
    "RMSE",
    "R2",
    "Peak90_RMSE",
    "Peak90_R2",
    "TOP5_RMSE",
    "TOP5_R2",
    "TOP5_Corr",
]
columns = [f"{set_name}_{metric}" for set_name in set_list for metric in metric_list]
columns.append("Augmented")

In [ ]:
# Prepare data
def prepare_data(file_path):
    """Prepares the dataset for training."""
    # Initialize data processor
    data_processor = RiverFlowDataProcessor(file_path)

    # Load and preprocess data
    data_processor.load_data()
    input_features, target_variable = data_processor.get_features_target()

    # Split the dataset
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = data_processor.split_data()
    print(
        f"Data split: Training set {len(X_train)}, Validation set {len(X_val)}, Test set {len(X_test)}"
    )

    # Augment peak flow data
    X_aug, y_aug = data_processor.augment_peak_flows()

    return (X_train, y_train), (X_val, y_val), (X_test, y_test), (X_aug, y_aug)


# Model training
def train_model(
    model_type, X_train, y_train, X_val, y_val, X_test, y_test, X_aug, y_aug
):
    """Trains the specified type of model."""
    # Initialize the model
    model = RiverFlowModel(model_type=model_type)

    # Feature selection
    selected_features = model.select_features(X_train, y_train)
    X_train_final = X_train[selected_features].copy()
    X_val_final = X_val[selected_features].copy()
    X_test_final = X_test[selected_features].copy()
    X_aug_final = X_aug[selected_features].copy()

    # Tune hyperparameters
    best_params = model.tune_hyperparameters(X_train_final, y_train)
    # print(f"\nBest parameters for {model_type} model:", best_params) # Original print statement, now commented out

    # Train the final model
    final_model, metrics = model.train_final_model(
        X_train_final,
        y_train,
        X_val_final,
        y_val,
        X_test_final,
        y_test,
        X_aug_final,
        y_aug,
    )
    metrics = pd.DataFrame(metrics, index=columns, columns=[model_type]).T
    metrics.index.name = "Model"
    metrics = metrics.round(3)

    return model, metrics

In [ ]:
file_data = PATH_DF + "Data2_without_nan.csv"
(X_train, y_train), (X_val, y_val), (X_test, y_test), (X_aug, y_aug) = prepare_data(
    file_data
)

model_types = [
    "XGBoost",  # XGBoost
    "LightGBM",  # LightGBM
    "GB",  # gradient boosting
    "RF",  # random forest
]

In [ ]:
results = []

# Train and evaluate each model
for model_type in model_types:
    print(f"Training {model_type} model...")

    model, metrics = train_model(
        model_type, X_train, y_train, X_val, y_val, X_test, y_test, X_aug, y_aug
    )
    metrics = metrics.round(3)
    results.append(metrics)

    print("\n\n\n-------------------------------------------------\n\n")

results = pd.concat(results)

In [ ]:
results.sort_values("Test_R2", ascending=False, inplace=True)
results

In [ ]:
results[["Train_R2", "Validation_R2", "Combined_R2", "Test_R2"]]

In [ ]:
results["Test_Peak90_R2"]

In [ ]:
results["Test_TOP5_R2"]

In [ ]:
results["Test_TOP5_Corr"]

In [ ]:
results["Overfitting"] = (results["Combined_R2"] - results["Test_R2"]) > 0.1

results["Score"] = results["Test_R2"] * 0.8 + results["Test_Peak90_R2"] * 0.2
results["Score"] = results["Score"].round(3)
results.sort_values("Score", ascending=False, inplace=True)
results.to_csv(PATH_DF + r"Metric\Model_metrics.csv")

In [ ]:
results